# <font color="red">**4. RF FS - High-freq Forecast**</font>

**Author:** Osmar Bolivar

In [194]:
#%pip install kaleido

In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go
import plotly.express as px

In [196]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


## **1. Monthly data**

In [197]:
raw = pd.read_csv("./Data/RF_DATASET.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Data columns (total 54 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ipc_all       178 non-null    float64
 1   lag_1         5901 non-null   float64
 2   lag_2         5901 non-null   float64
 3   lag_3         5901 non-null   float64
 4   lag_6         5901 non-null   float64
 5   lag_9         5901 non-null   float64
 6   lag_12        5901 non-null   float64
 7   oil_cb        5901 non-null   float64
 8   sugar_bol     5901 non-null   float64
 9   chicken_tj    5901 non-null   float64
 10  oil2_tj       5901 non-null   float64
 11  oil2_bol      5901 non-null   float64
 12  sugar_tj      5901 non-null   float64
 13  rice4_tj      5901 non-null   float64
 14  sugar_cb      5901 non-null   float64
 15  rice3_po      5901 non-null   float64
 16  sugar_or      5901 non-null   float64
 17  sugar_sc      5901 non-null   float64
 18  rice4_su      5901

In [198]:
isna = raw.isna().sum()
isna[isna > 0]

ipc_all    5723
dtype: int64

In [199]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)

df_w = raw.copy().query('freq == "week"')
df_w.drop(columns=["freq"], inplace=True)

df_d = raw.copy().query('freq == "day"')
df_d.drop(columns=["freq"], inplace=True)

In [200]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [201]:
df.shape

(178, 53)

In [202]:
isna_d = df_d.isna().sum()
isna_d[isna_d > 0]

ipc_all    5418
dtype: int64

## **2. Train and Val sets**

In [203]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [204]:
#train_size = int(len(df) * 0.8)
#train, validation = df[:train_size], df[train_size:]
train, validation = train_test_split(df, test_size=0.2, random_state=42)

print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 53
Obs in validation set: 36; variables in validation set: 53


In [205]:
test_w = df_w.copy()
test_d = df_d.copy()
print(f'Obs in weekly test set: {test_w.shape[0]}; variables in weekly test set: {test_w.shape[1]}')
print(f'Obs in daily test set: {test_d.shape[0]}; variables in daily test set: {test_d.shape[1]}')

Obs in weekly test set: 305; variables in weekly test set: 53
Obs in daily test set: 5418; variables in daily test set: 53


In [206]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
test_scaled_w = pd.DataFrame(scaler.transform(test_w), columns=test_w.columns, index=test_w.index)
test_scaled_d = pd.DataFrame(scaler.transform(test_d), columns=test_d.columns, index=test_d.index)

X_train = train_scaled.drop('ipc_all', axis=1)
y_train = train['ipc_all']
#y_train = train_scaled['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
y_validation = validation['ipc_all']
#y_validation = validation_scaled['ipc_all']

X_test_w = test_scaled_w.drop('ipc_all', axis=1)
X_test_d = test_scaled_d.drop('ipc_all', axis=1)

## **3. Algorithms**

In [207]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  2.209; R2:  0.001; MAE:  1.256

In [208]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=3.5, fit_intercept=True, positive=False, random_state=0)  
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

ridge_test_pred_w = ridge.predict(X_test_w)
ridge_test_pred_d = ridge.predict(X_test_d)

Train MSE:  0.12150902144125023
Train R2:  0.9772590403414474
Train MAE:  0.2677449348458906
Validation MSE:  0.1983560684158458
Validation R2:  0.9783978379053767
Validation MAE:  0.332231035304865


In [209]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,1.011058
1,rice3_su,0.310318
2,oil_cb,0.301275
3,flour_lp,0.277122
4,lag_6,0.231458
5,oil2_bol,0.199587
6,sugar_lp,0.187367
7,silver,0.184285
8,squash_co,0.167040
9,platano_co,0.141228


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.796
Validation R2:  0.640
Validation MAE:  0.690

In [210]:
#lasso = Lasso(random_state=0)
lasso = Lasso(alpha=0.37, fit_intercept=True, max_iter=10000, random_state=0)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

lasso_test_pred_w = lasso.predict(X_test_w)
lasso_test_pred_d = lasso.predict(X_test_d)

Train MSE:  0.5032397538166109
Train R2:  0.9058164175434764
Train MAE:  0.5177590474885496
Validation MSE:  0.6489205348297913
Validation R2:  0.9293286729673708
Validation MAE:  0.6140107022298872


In [211]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,1.853639
1,sugar_lp,0.010901
2,milk_sc,-0.000000
3,pineapple_tr,-0.000000
4,rice2_lp,0.000000
5,rice3_su,0.000000
6,ufv,-0.000000
7,paprika_or,-0.000000
8,squash_co,0.000000
9,papa1_po,0.000000


### **3.4. ADA**
Without tuning:
Validation MSE:  0.324
Validation R2:  0.853
Validation MAE:  0.429

In [212]:
#ada = AdaBoostRegressor(random_state=0)
ada = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=5), loss='square', learning_rate=0.98, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

ada_test_pred_w = ada.predict(X_test_w)
ada_test_pred_d = ada.predict(X_test_d)

Train MSE:  0.033663357225945975
Train R2:  0.9936997513471308
Train MAE:  0.13195404384265325
Validation MSE:  0.5398822667655332
Validation R2:  0.9412035924495565
Validation MAE:  0.5628845137761391


In [213]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
feature_importance_ada.head(15)

,feat,imp_ada
0,lag_1,0.776651
1,sugar_cb,0.038340
2,sugar_tj,0.038333
3,silver,0.020383
4,lag_2,0.018832
5,chicken_tj,0.010450
6,chicken_bol,0.010219
7,squash_co,0.007850
8,rice2_po,0.006871
9,pineapple_tr,0.006656


### **3.5. GBR**
Without tuning:
Validation MSE:  0.277
Validation R2:  0.874
Validation MAE:  0.414

In [214]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(learning_rate=0.5, n_estimators=110, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

gbr_test_pred_w = gbr.predict(X_test_w)
gbr_test_pred_d = gbr.predict(X_test_d)

Train MSE:  8.239624313561905e-08
Train R2:  0.9999999845791726
Train MAE:  0.0002291646163215954
Validation MSE:  0.3676643868407951
Validation R2:  0.9599591495012679
Validation MAE:  0.4775748903631116


In [215]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,lag_1,0.760341
1,chicken_tj,0.115432
2,flour_lp,0.040767
3,sugar_cb,0.030411
4,sugar_lp,0.007678
5,tomato_tj,0.006752
6,chicken_sc,0.004925
7,lag_6,0.003984
8,ycorn_tj,0.003744
9,papa1_po,0.003726


### **3.4. RF**
Without tuning:
Validation MSE:  0.324
Validation R2:  0.853
Validation MAE:  0.434

In [216]:
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(max_depth=6, criterion="absolute_error", random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

rf_test_pred_w = rf.predict(X_test_w)
rf_test_pred_d = rf.predict(X_test_d)

Train MSE:  0.05589833513197085
Train R2:  0.9895383752651566
Train MAE:  0.17251127385657897
Validation MSE:  0.5268396094388966
Validation R2:  0.9426240158324396
Validation MAE:  0.5370573762746291


In [217]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
feature_importance_rf.head(30)

,feat,imp_rf
0,lag_1,0.700666
1,papa1_po,0.018119
2,banana_co,0.016273
3,chicken_tj,0.013458
4,paprika_or,0.013409
5,sugar_tj,0.012029
6,tomato_tj,0.011601
7,pineapple_tr,0.011538
8,ycorn_sc,0.010308
9,ycorn_tj,0.010259


### **3.5. ET**   
Without tuning:
Validation MSE:  0.322
Validation R2:  0.854
Validation MAE:  0.423

In [218]:
#et = ExtraTreesRegressor(random_state=0)
et = ExtraTreesRegressor(max_depth=9, criterion='absolute_error', bootstrap=True, oob_score=True, random_state=0)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

et_test_pred_w = et.predict(X_test_w)
et_test_pred_d = et.predict(X_test_d)

Train MSE:  0.046772046537093326
Train R2:  0.9912464012068252
Train MAE:  0.15493602248628555
Validation MSE:  0.37988479623600985
Validation R2:  0.9586282738354696
Validation MAE:  0.48030506282310614


In [219]:
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
feature_importance_et.head(30)

,feat,imp_et
0,lag_1,0.244389
1,lag_2,0.069253
2,lag_3,0.040238
3,papa2_tj,0.038362
4,sugar_tj,0.026900
5,oil2_po,0.026407
6,chicken_tj,0.025496
7,ycorn_tj,0.023171
8,squash_co,0.022081
9,oil_cb,0.021078


## **4. Report**

In [220]:
val_forecast = pd.DataFrame(
    {'cpi': y_validation,
     'ridge': ridge_val_pred,
     'lasso': lasso_val_pred,
     'ada': ada_val_pred,
     'gbr': gbr_val_pred,
     'rf': rf_val_pred,
     'et': et_val_pred
    }, index=validation.index
)
val_forecast['set'] = 'validation'

week_forecast = pd.DataFrame(
    {'cpi': np.nan,
     'ridge': ridge_test_pred_w,
     'lasso': lasso_test_pred_w,
     'ada': ada_test_pred_w,
     'gbr': gbr_test_pred_w,
     'rf': rf_test_pred_w,
     'et': et_test_pred_w
    }, index=test_w.index
)
week_forecast['set'] = 'week'

day_forecast = pd.DataFrame(
    {'cpi': np.nan,
     'ridge': ridge_test_pred_d,
     'lasso': lasso_test_pred_d,
     'ada': ada_test_pred_d,
     'gbr': gbr_test_pred_d,
     'rf': rf_test_pred_d,
     'et': et_test_pred_d
    }, index=test_d.index
)
day_forecast['set'] = 'day'

all_forecast = pd.concat([val_forecast, week_forecast, day_forecast], axis=0)
all_forecast = all_forecast.sort_index(ascending=True)
all_forecast.tail(40)

,cpi,ridge,lasso,ada,gbr,rf,et,set
2024-09-27,NaN,6.950527,4.905719,6.182067,7.752051,6.229523,6.204402,day
2024-09-28,NaN,6.939665,4.905719,6.182067,7.482702,6.232150,6.204402,day
2024-09-29,NaN,6.846430,4.906257,6.182067,7.593435,6.217611,6.208832,week
2024-09-29,NaN,6.935918,4.905719,6.182067,7.482702,6.232150,6.204402,day
2024-09-30,NaN,6.871828,4.905719,6.182067,7.428839,6.127597,6.204224,day
2024-10-01,NaN,7.508810,5.708265,7.938964,7.901042,7.396616,6.814092,day
2024-10-02,NaN,7.512460,5.708265,7.938964,7.939133,7.400981,6.892739,day
2024-10-03,NaN,7.578930,5.708265,7.938964,7.939133,7.394568,6.966788,day
2024-10-04,NaN,7.854830,5.708265,7.938964,7.939133,7.400612,7.019115,day
2024-10-05,NaN,7.848830,5.708265,7.938964,7.939133,7.400612,6.972526,day


In [221]:
metrics = pd.DataFrame(
    {'MSE': [mse_val_ridge, mse_val_lasso, mse_val_ada, mse_val_gbr, mse_val_rf, mse_val_et],
     'R2': [r2_val_ridge, r2_val_lasso, r2_val_ada, r2_val_gbr, r2_val_rf, r2_val_et],
     'MAE': [mae_val_ridge, mae_val_lasso, mae_val_ada, mae_val_gbr, mae_val_rf, mae_val_et]},
    index=['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']
)
metrics.sort_values(['MSE'], ascending=True, inplace=True)
metrics

,MSE,R2,MAE
ridge,0.198356,0.978398,0.332231
gbr,0.367664,0.959959,0.477575
et,0.379885,0.958628,0.480305
rf,0.526840,0.942624,0.537057
ada,0.539882,0.941204,0.562885
lasso,0.648921,0.929329,0.614011


In [222]:
all_forecast['min'] = all_forecast[['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']].min(axis=1)
all_forecast['max'] = all_forecast[['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']].max(axis=1)

inv1 = (1/mse_val_ridge)  / 1000
inv2 = (1/mse_val_lasso) / 1000
inv3 = (1/mse_val_ada) / 1000
inv4 = (1/mse_val_gbr) / 1000
inv5 = (1/mse_val_rf) / 1000
inv6 = (1/mse_val_et) / 1000

num_1 = (all_forecast['ridge'] * inv1) + (all_forecast['lasso'] * inv2) + (all_forecast['ada'] * inv3) + (all_forecast['gbr'] * inv4) + (all_forecast['rf'] * inv5) + (all_forecast['et'] * inv6)
den_1 = inv1+inv2+inv3+inv4+inv5+inv6
all_forecast['w_avg'] = num_1 / den_1

num_2 = (all_forecast['ridge'] * inv1) + (all_forecast['gbr'] * inv4) + (all_forecast['et'] * inv6)
den_2 = inv1+inv4+inv6
all_forecast['w_avg_best'] = num_2 / den_2

all_forecast

,cpi,ridge,lasso,ada,gbr,rf,et,set,min,max,w_avg,w_avg_best
2010-01-01,NaN,-0.708331,0.883229,0.074472,-0.110377,0.137302,0.427570,day,-0.708331,0.883229,-0.062866,-0.264169
2010-01-02,NaN,-0.692633,0.883229,0.141549,-0.111972,0.125392,0.416625,day,-0.692633,0.883229,-0.053454,-0.259745
2010-01-03,NaN,-0.690356,0.883229,0.141549,-0.111972,0.128774,0.416625,day,-0.690356,0.883229,-0.052313,-0.258640
2010-01-04,NaN,-0.880319,0.882978,0.074472,-0.248228,0.137263,0.409595,day,-0.880319,0.882978,-0.145096,-0.388218
2010-01-05,NaN,-0.944976,0.882978,0.074472,-0.246633,0.137039,0.414895,day,-0.944976,0.882978,-0.164739,-0.417820
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27,NaN,8.064767,5.707637,7.938964,7.939241,7.482332,7.239117,week,5.707637,8.064767,7.587513,7.822809
2024-10-28,NaN,8.055166,5.708265,7.938964,8.751872,7.700411,7.378243,day,5.708265,8.751872,7.775143,8.066041
2024-10-29,NaN,7.959163,5.708265,7.938964,8.283737,7.700411,7.378719,day,5.708265,8.283737,7.663188,7.897092
2024-10-30,NaN,8.258877,5.708265,7.938964,8.283737,7.730141,7.337225,day,5.708265,8.283737,7.756156,8.031959


In [223]:
all_forecast['cpi'] = df['ipc_all'].reindex(all_forecast.index)
all_forecast.tail(35)

,cpi,ridge,lasso,ada,gbr,rf,et,set,min,max,w_avg,w_avg_best
2024-10-01,NaN,7.508810,5.708265,7.938964,7.901042,7.396616,6.814092,day,5.708265,7.938964,7.320554,7.435502
2024-10-02,NaN,7.512460,5.708265,7.938964,7.939133,7.400981,6.892739,day,5.708265,7.939133,7.342059,7.467159
2024-10-03,NaN,7.578930,5.708265,7.938964,7.939133,7.394568,6.966788,day,5.708265,7.939133,7.375075,7.518155
2024-10-04,NaN,7.854830,5.708265,7.938964,7.939133,7.400612,7.019115,day,5.708265,7.939133,7.473267,7.665232
2024-10-05,NaN,7.848830,5.708265,7.938964,7.939133,7.400612,6.972526,day,5.708265,7.939133,7.463520,7.650522
2024-10-06,NaN,7.844470,5.708265,7.938964,7.939133,7.400612,6.972526,day,5.708265,7.939133,7.462118,7.648407
2024-10-06,NaN,7.667685,5.708265,7.938964,7.901042,7.399454,6.863862,week,5.708265,7.938964,7.380315,7.525169
2024-10-07,NaN,7.872279,5.708265,7.938964,7.939133,7.449399,7.056359,day,5.708265,7.939133,7.491030,7.683128
2024-10-08,NaN,7.849233,5.708265,7.938964,7.939133,7.461318,7.128087,day,5.708265,7.939133,7.497103,7.690116
2024-10-09,NaN,7.833374,5.708265,7.938964,7.939133,7.449570,7.152725,day,5.708265,7.939133,7.494719,7.688664


In [224]:
only_validation = all_forecast.query('set == "validation"')

mse_val_w_avg = mean_squared_error(only_validation['cpi'], only_validation['w_avg'])
r2_val_w_avg = r2_score(only_validation['cpi'], only_validation['w_avg'])
mae_val_w_avg = mean_absolute_error(only_validation['cpi'], only_validation['w_avg'])

mse_val_w_avg_best = mean_squared_error(only_validation['cpi'], only_validation['w_avg_best'])
r2_val_w_avg_best = r2_score(only_validation['cpi'], only_validation['w_avg_best'])
mae_val_w_avg_best = mean_absolute_error(only_validation['cpi'], only_validation['w_avg_best'])

metrics_b = pd.DataFrame(
    {'MSE': [mse_val_w_avg, mse_val_w_avg_best],
     'R2': [r2_val_w_avg, r2_val_w_avg_best],
     'MAE': [mae_val_w_avg, mae_val_w_avg_best]},
    index=['w_avg', 'w_avg_best']
)

metrics_all = pd.concat([metrics, metrics_b], axis=0)
metrics_all.sort_values(['MSE'], ascending=True, inplace=True)
metrics_all

,MSE,R2,MAE
ridge,0.198356,0.978398,0.332231
w_avg_best,0.213436,0.976755,0.369332
w_avg,0.256219,0.972096,0.406840
gbr,0.367664,0.959959,0.477575
et,0.379885,0.958628,0.480305
rf,0.526840,0.942624,0.537057
ada,0.539882,0.941204,0.562885
lasso,0.648921,0.929329,0.614011


In [225]:
# Create a scatter plot
fig = px.scatter(only_validation, x='cpi', y='ridge')
# Fit a line to the data
fit = np.polyfit(only_validation['cpi'], only_validation['ridge'], deg=1)
fit_fn = np.poly1d(fit)
# Add the fitted line to the plot
fig.add_trace(go.Scatter(
    x=only_validation['cpi'], 
    y=fit_fn(only_validation['cpi']), 
    mode='lines', 
    name='Fitted Line',
    line=dict(color='red', width=2)
))

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Observed y-o-y Inflation',
    yaxis_title='Best Forecast',
    template='plotly_white',
    width=800, 
    height=600,
    legend=dict(
        x=0.8,
        y=0.1,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='black',
        borderwidth=1
    ),
    title=dict(
        x=0.5,
        xanchor='center'
    )
)

# Show the plot
fig.show()

In [226]:
weekly_forecast = all_forecast.copy().query('set == "week"')
daily_forecast = all_forecast.copy().query('set == "day"')

#### Weekly Forecast

In [227]:
fig1 = go.Figure()
# Add the forecast interval fill
fig1.add_trace(go.Scatter(
    x=weekly_forecast.index.tolist() + weekly_forecast.index[::-1].tolist(),
    y=weekly_forecast["min"].tolist() + weekly_forecast["max"][::-1].tolist(),
    fill='toself',
    fillcolor='#ffdabe',
    line=dict(color='rgba(255,255,255,0.5)'),
    hoverinfo="skip",
    showlegend=True,
    name="Forecast Interval"
))
# Add the high-frequency forecast line
fig1.add_trace(go.Scatter(
    x=weekly_forecast.index, 
    y=weekly_forecast["ridge"], 
    mode='lines+markers', 
    name="Weekly Forecast", 
    line=dict(color='#ff7d00', width=1.0),
    marker=dict(size=2.5)
))
# Add the y-o-y inflation line
fig1.add_trace(go.Scatter(
    x=df.loc['2018-12-30':, ].index, 
    y=df.loc['2018-12-30':, "ipc_all"], 
    mode='markers', 
    name="Observed y-o-y Inflation", 
    line=dict(color='#073763', width=0.8),
    marker=dict(size=4)
))
# Update layout for better visualization
fig1.update_layout(
    #title='High-frequency Inflation Forecast and y-o-y Inflation',
    #xaxis_title='Date',
    #yaxis_title='%',
    #legend_title='Legend',
    template='plotly_white',
    width=1200, 
    height=600,
    legend=dict(
        x=0.6,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        font=dict(size=15),
        bordercolor='black',
        borderwidth=1
    )
)
# Update x-axis to show dates in the desired format
fig1.update_xaxes(tickformat='%b %Y')
# Show the plot
fig1.show()

# Save the plot as an image
fig1.write_image('./Images/F4_Weekly_Forecast.png', format='png', scale=3)

#### Daily Forecast

In [228]:
fig2 = go.Figure()
# Add the forecast interval fill
fig2.add_trace(go.Scatter(
    x=daily_forecast.index.tolist() + daily_forecast.index[::-1].tolist(),
    y=daily_forecast["min"].tolist() + daily_forecast["max"][::-1].tolist(),
    fill='toself',
    fillcolor='#ffdabe',
    line=dict(color='rgba(255,255,255,0.5)'),
    hoverinfo="skip",
    showlegend=True,
    name="Forecast Interval"
))
# Add the high-frequency forecast line
fig2.add_trace(go.Scatter(
    x=daily_forecast.index, 
    y=daily_forecast["w_avg_best"], 
    mode='lines', 
    name="Daily Forecast", 
    line=dict(color='#ff7d00', width=1.0),
    #marker=dict(size=1.5)
))
# Add the y-o-y inflation line
fig2.add_trace(go.Scatter(
    x=df.index, 
    y=df["ipc_all"], 
    mode='markers', 
    name="Observed y-o-y Inflation", 
    line=dict(color='#073763', width=0.8),
    marker=dict(size=4)
))
# Update layout for better visualization
fig2.update_layout(
    #title='High-frequency Inflation Forecast and y-o-y Inflation',
    #xaxis_title='Date',
    #yaxis_title='%',
    #legend_title='Legend',
    template='plotly_white',
    width=1200, 
    height=600,
    legend=dict(
        x=0.75,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        font=dict(size=15),
        bordercolor='black',
        borderwidth=1
    )
)
# Update x-axis to show dates in the desired format
fig2.update_xaxes(tickformat='%b %Y')
# Show the plot
fig2.show()

# Save the plot as an image
fig2.write_image('./Images/F4_Daily_Forecast.png', format='png', scale=3)

In [229]:
all_forecast.to_csv('./Forecast/F4_FORECAST.csv')

#### End